<a href="https://colab.research.google.com/github/priyansh-shankhdhar/interviews/blob/master/hope_this_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# in this at first
create/initilise the class as suppose
>d=intrview()
#now we create the word to vec; //this takes some time like a minute or 3 
>w=d.run()
#because d.run() takes some time we dont have to run it again and again so, we can use
>w=d.run(w) #here w is from previous d.run()
#now in order to predict interview use
>d.pred(dataset)
# but the dataset has to be similar to the format in jobs.csv or train_it.csv
# otherwise it won't make predictions and throw errors



In [ ]:
import numpy as np
import pandas as pd
import regex as re
#from google.colab import files
import os
import zipfile
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from gensim.models import Word2Vec
nltk.download('punkt')
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer
import random

[nltk_data] Downloading package stopwords to C:\Users\Manna's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Manna's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
class wordy:
  def __init__(self,data):
    self.model=None
    self.data=data

  def create_word_to_vec(self):

    if self.model is None:
      print('word to vec forming: ')

      data=self.data #pd.read_csv('/content/more/marketing_sample_for_naukri_com-jobs__20190701_20190830__30k_data.csv')

      wv=data[['Job Title','Key Skills','Role Category','Functional Area','Industry','Role']]

      wv=wv.fillna('')
      
      wv['Key Skills']=wv['Key Skills'].apply(lambda x: str(x).replace('|',''))
      
      wv['Functional Area']=wv['Functional Area'].apply(lambda x: x.replace(',',''))
      
      wv['Industry']=wv['Industry'].apply(lambda x: x.replace(',',''))

      print('==',end='>')

      arr=[]
      for i in range(len(wv)):
        x=(wv.iloc[i][0]+' '+wv.iloc[i][1]+' '+wv.iloc[i][2]+' '+wv.iloc[i][3]+' '+wv.iloc[i][4]+' '+wv.iloc[i][5]).lower()
        x=re.sub('\[[0-9]"\]',' ',x)
        x=re.sub('\d',' ',x)
        x=re.sub('\s+',' ',x)
        arr.append(x)
      print('==',end='>')
      sentences=[nltk.word_tokenize(i) for i in arr]
      print('==',end='>')
      for i in range(len(sentences)):
        sentences[i]=[word for word in sentences[i] if word not in stopwords.words('english')]
      print('==',end='>')
      self.model=Word2Vec(sentences,min_count=1,size=1)
      ########################################################### CAN BE DELETED JUST FOR SHOW--down
      print('=>\ncompleted')

    else:
      #print('word to vector has already formed')
      return True

  def get_vec(self,word=''):
    if self.model is None:
      return ('model not formed please call: wordy.create_word_to_vec() ')    
    try:
      x=self.model.wv[word]
      return x[0]
    except:
      return 0.5

  def get_model(self):
    if self.model:
      return self.model
    else:
      print('model not formed please call: wordy.create_word_to_vec()')
#***********************************************************************************************************************





def filter_data(df,w,city):
  df=df[['Salary per Month','Job title','Year of experience','Job location','About job','status','Number of openings','Requirements','Qualification']]

  #########################################################################
  df['Salary per Month']=df['Salary per Month'].apply(lambda x: float(str(x).replace('k','000').partition('-')[0])/1000 )

  ########################################################################
  def ch(x):
    try:
        x=x.split('-')
        x="".join(x)
        x=re.sub('\D',"",x)
        return float(x)
    except:
        return float(x)

  df['Year of experience']=df['Year of experience'].apply(lambda x: ch(x))

  #########################################################################
  d={ 'open':1.0, 'close': 0.0 ,'':-1.0}
  df=df.replace({'status': d})

  #################################################
  def qual(x):
    try:
        return float(x.count(','))+1
    except:
        return x
        

  df['Qualification']=df['Qualification'].apply(lambda x: qual(x))

  ###############################################################################################

  #clean the city.csv file 
  city=city.drop(['State','Type','Population class','Type'],axis=1)
  city['Name of City']=city['Name of City'].apply(lambda x: x.lower())
  city['Population (2011)']=city['Population (2011)'].apply(lambda x: float(re.sub(',','',x)))
  # assinging value sto each city w.r.t the population of the city and storing in  a dictionary
  k=[]
  dic={}
  cit_lis=[]
  for i in range(len(city)):
    v=city.iloc[i][1]
    if v >= 10000:
      m= 4
    if v >= 20000:
      m=3
    if v >= 50000:
      m=2
    if v >= 100000:
      m=1
    else :
      m=5
    cit_lis.append(city.iloc[i][0])
    dic.update({city.iloc[i][0] : m })

  def jo(x):
    try:
        x=x.split(',')
        sum=0
        t=[]
        for i in x:
          p=i.split('(')[0].strip(' ')
          t.append(p)
        for i in t:
          for j in i.split():
            j=j.lower()
            if j in cit_lis:
              sum+=dic[j]
        if sum==0:
          return 10
    except:
          return x
  # here we are using this such that the value does no ecceed or decrease a limit we want it to be in a reage
  # the equation of given fuction below can be seen here-> https://www.google.com/search?rlz=1C1CHBD_enIN894IN895&sxsrf=ALeKk023pu9vQvanAQYWZ0dpLw-2Ug1QWw%3A1599882619675&ei=e0VcX_bhKPid4-EPj5KXyAs&q=y%3D20%2Fx&oq=y%3D20%2Fx&gs_lcp=CgZwc3ktYWIQAzICCAAyBAgAEB4yBAgAEB4yBAgAEB4yBAgAEB4yBAgAEB4yBAgAEB4yBAgAEB4yBggAEAUQHjIGCAAQBRAeOgcIIxDqAhAnOgQIABBDOggIABCxAxCDAToFCAAQsQM6BAgjECc6AggmOgYIABAKEB5QubafAliR2p8CYMPhnwJoAXAAeACAAecCiAGDCZIBBzAuNS4wLjGYAQCgAQGqAQdnd3Mtd2l6sAEKwAEB&sclient=psy-ab&ved=0ahUKEwi2i-mZ2-LrAhX4zjgGHQ_JBbkQ4dUDCA0&uact=5
    return (20/sum)-len(t)

  ################################################################
  df['Job location']=df['Job location'].apply(lambda x: jo(x))
  #############################################################
  # Now for job title and about job i have created a word to vec from a preexisting dataset from naukri.com
  #with that information once the word to vect for specific word was created (here ecah vecto in 1 dimesional)
  #we replace the word in 'job title' and 'about job' woth their specific vector and if vector is not available
  #the surely it means the work is not much in demand thus the vector we get is zer

  #so for multiole words in a block we take avergae of vectors of each word in block in do_it(x) funtion 

  #w=wordy() #load the wordy class # do not uncomment w=wordy()
  w.create_word_to_vec() #this creates the word to vector

  #this function averages out the vector value for each block
  def do_it(x):
    try:
        x=x.lower().split()
        c=0
        sum=0
        for j in x:
          c+=1
          sum+=w.get_vec(j)#here w.do(j) gets the vector for string j
        return sum/c 
    except:
        return float(x)

  df['Job title']=df['Job title'].apply(lambda x: do_it(x))

  ###################################################################


  for i in range(len(df['About job'])):
    sentence=df['About job'][i]
    try:
        sentence=str(sentence.replace('  ','')).split(',')

        for i in range(len(sentence)):
          sentence[i]=sentence[i].replace('  ','').replace('.','').lower().split() 
          sentence[i]=[word for word in sentence[i] if word not in stopwords.words('english') ] 
          sentence[i]=' '.join(sentence[i]) 
          df['About job'][i]=sentence[i] 
    except:
        pass

  df['About job']=df['About job'].apply(lambda x: do_it(x))

  ####################################################
  def req(x):
    try:
        di={'they would have to stay near our plant':4 ,'no':0}
        x=x.lower()
        try:
          return di[x]
        except:
          return -2
    except:
        return x

  df['Requirements']=df['Requirements'].apply(lambda x: req(str(x)))

  columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [1])], 
                                      remainder='passthrough')
  h=pd.DataFrame(columnTransformer.fit_transform(df), dtype = np.str) 

  return h
#**************************************************************************************************************




def unzip(path, dir_name):
  with zipfile.ZipFile(path, 'r') as zip_ref:
      os.mkdir(dir_name)
      j=%pwd
      zip_ref.extractall(os.path.join(j,dir_name))

def load_data():
  #data_to_load = files.upload()
  j=input('enter location of directory in which files are saved: ')
  try:
    try:
      unzip(j+"\data.zip",'more')
    except:
      pass
    data=pd.read_csv(j+"\more\marketing_sample_for_naukri_com-jobs__20190701_20190830__30k_data.csv")
    df=pd.read_csv(j+'\job.csv')
    city=pd.read_csv(os.path.join(j+'\city_lis.csv'))
    return data,df,city
  except:
    print("You Dont have the files get the files in same directory")


def wording(data):
  w=wordy(data)  
  w.create_word_to_vec()
  return w

def calls(h):
  #print(h)
  scalar = MinMaxScaler()
  scaled_data = scalar.fit_transform(h)
  km=KMeans(n_clusters=5,max_iter=2000,random_state=85)
  km=km.fit(scaled_data)
  return km.labels_

#import_()
#data,df,city=load_data()
#w=wording(data)
#h=filter_data(df,w,city)
#op=calls(h)

class interview:
  def __init__(self):
    self.city=None
    self.w=None
    self.h=None

  def run(self,w=None):
    data,df,self.city = load_data()
    #print(data,df,self.city)
    self.h=df=df[['Salary per Month','Job title','Year of experience','Job location','About job','status','Number of openings','Requirements','Qualification']].loc[0:7]
    if w is None:
        self.w=wording(data)
        return self.w
    else:
        self.w=w
        return self.w

  def pred(self,dframe,filt=True):
    hh=self.h
    hh=hh.append(dframe,ignore_index=True)
    hh=filter_data(hh,self.w,self.city)
    l=calls(hh)
    ep=[]
    print(l[8:])
    for i in range(len(dframe)):
      if l[8+i]==4 or l[8+i]==1:
        ep.append(np.array(dframe.loc[i]))
    return pd.DataFrame(np.array(ep))


In [ ]:
#not necessary though
data,df,city=load_data()

enter directory in which files are saved: C:\Users\Manna's PC\Downloads


In [ ]:
testing=pd.read_csv(r"C:\Users\Manna's PC\Downloads\test_it.csv")
testing.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
d=interview()
w=d.run()

In [ ]:
d.pred(testing)

[3 4 3 3 3 4 2 2 2 2 4 4 3 2 2 3 2 4 4 2 3 4 2 1 1 3 3 4 1 3 1 2 1 3 2 3 2
 4 1 3 2 4 1 1 3 1 1 3 2 2 4 3 1 1 1 3 4 4 3 3 4 2 2 2 2 2 4 4 3 2 2 3 2 4
 2 4 2 3 4 2 1 4 1 3 3 4 1 2 3 1 2 1 3 2 3 2 4 1 3 2 4 1 1 3 1 1 3 2 2 4 3
 1 1 3 1 2]


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,0,1,2,3,4,5,6,7,8
0,0,-4.87208,0,6,1.3657,0.0,20,9.21947,1
1,0,-0.908291,1,10,-1.31281,close,15,9.68951,1
2,0,-4.83065,0,8,-0.771391,0.0,5,4.27025,1
3,0,-4.87208,0,6,1.48978,0.0,10,1.41637,1
4,0,-1.20327,3,6,-1.33898,close,10,6.03224,1
5,0,-1.96073,0,6,1.06177,0.0,5,3.11725,1
6,0,-1.06879,2,5,0.572265,close,20,7.78809,1
7,15,0.5,0,9,-0.605034,open,5,4.20065,1
8,5,0.5,2,0.8,1.19001,open,20,6.44616,1
9,0,-4.65688,0,8,0.388097,0.0,5,1.38965,1


In [ ]:
#anotherr prediction
d.pred(df)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[0 1 4 2 0 1 2 0 3 3]


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,"Timestamp(seconds=1594915696, nanoseconds=8281...",12,3w4OH4ITykUZEXLXYi2cz8UCPcx2,8000-12000,https://firebasestorage.googleapis.com/v0/b/fr...,Kitchen Helper,0-3,"Mumbai, Pune,Hyderabad, Delhi","['A', 'M', 'K']",Preparing & cooking meals and helping the main...,...,01-08-2020,open,10,Absolute Barbecue,8f10efc0-c77e-11ea-d1fb-bbad64001e7f,"Experience of South Indian, Continental food",8th pass,8S5GyR2lGacUmTa1jsmv,NaN,NaN
1,"Timestamp(seconds=1594897761, nanoseconds=6679...",24,DFnrjr0KMFf5VdFuerlzDGOtBei2,10000-15000,https://firebasestorage.googleapis.com/v0/b/fr...,Assemby Operators,0-1,"Bawal (Haryana), Ranjangaon near Pune (Maharas...","['P', 'B', 'A']",to put products and product components togethe...,...,01-08-2020,open,15,Polyplastics Industries India Pvt.Ltd.,cce08d30-c754-11ea-f016-295b48f42895,no,"ITI, Diploma",JSAMoPj2NKLhVm9geEqT,NaN,NaN
2,"Timestamp(seconds=1594915515, nanoseconds=7355...",24,3w4OH4ITykUZEXLXYi2cz8UCPcx2,9000-12000,https://firebasestorage.googleapis.com/v0/b/fr...,Waiter,01-Feb,"Pune,Mumbai, Hyderabad, Delhi","['A', 'P', 'W']","To serve dish, helps with cooking & cleaning",...,01-08-2020,open,10,Absolute Barbecue,23205670-c77e-11ea-9322-23abb90748ec,no,"Proficient in Hindi,Marathi, 10th pass",Sc35yz2EI4qBl8CST2fX,NaN,NaN
